In [6]:
import os

import numpy as np
import pandas as pd

from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline

/Users/akihiro/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
event_prefix = 'event000001000'
hits, cells, particles, truth = load_event(os.path.join('../input/train_1', event_prefix))

mem_bytes = (hits.memory_usage(index=True).sum() 
             + cells.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())
print('{} memory usage {:.2f} MB'.format(event_prefix, mem_bytes / 2**20))

event000001000 memory usage 18.46 MB


In [5]:
hits.describe()

,hit_id,x,y,z,volume_id,layer_id,module_id
count,120939.00000,120939.000000,120939.000000,120939.000000,120939.000000,120939.000000,120939.000000
mean,60470.00000,-1.769457,4.751043,-2.314452,10.921564,5.918389,371.969117
std,34912.22644,305.714539,305.173279,1061.935303,3.365705,3.407428,529.032206
min,1.00000,-1024.839966,-1025.109985,-2955.500000,7.000000,2.000000,1.000000
25%,30235.50000,-100.146000,-95.896503,-655.799988,8.000000,4.000000,60.000000
50%,60470.00000,-1.651340,1.005920,-1.800000,9.000000,6.000000,119.000000
75%,90704.50000,97.267651,103.224003,655.799988,13.000000,8.000000,490.000000
max,120939.00000,1025.329956,1024.849976,2955.500000,18.000000,14.000000,3192.000000


In [7]:
cells.describe()

,hit_id,ch0,ch1,value
count,664996.000000,664996.000000,664996.000000,6.649960e+05
mean,49603.957174,211.529248,444.395715,3.560307e-01
std,32415.296519,156.399517,411.081790,4.322301e-01
min,1.000000,0.000000,0.000000,1.461840e-07
25%,25174.000000,97.000000,55.000000,5.364857e-02
50%,36148.000000,191.000000,340.000000,6.512935e-02
75%,78196.000000,287.000000,802.000000,1.000000e+00
max,120939.000000,1195.000000,1279.000000,1.000000e+00


In [8]:
particles.describe()

,particle_id,vx,vy,vz,px,py,pz,q,nhits
count,1.226300e+04,12263.000000,12263.000000,12263.000000,12263.000000,12263.000000,12263.000000,12263.000000,12263.000000
mean,4.803273e+17,1.107619,0.767672,7.498722,-0.003691,0.000103,0.100067,0.107233,8.424121
std,2.983347e+17,124.100189,124.626076,537.843567,0.930212,0.640032,7.765715,0.994274,5.033464
min,4.503668e+15,-955.513977,-901.664978,-2955.500000,-45.300701,-16.516300,-95.573402,-1.000000,0.000000
25%,2.161771e+17,-0.017552,-0.013040,-5.444660,-0.215898,-0.208301,-0.815998,-1.000000,4.000000
50%,4.548682e+17,0.000457,0.002574,-0.438758,-0.005679,0.006612,0.001188,1.000000,10.000000
75%,7.431018e+17,0.014919,0.013532,6.016730,0.203908,0.215444,0.848551,1.000000,12.000000
max,9.683079e+17,968.200012,980.807007,2950.000000,53.201599,11.169400,459.528015,1.000000,19.000000


In [9]:
truth.describe()

,hit_id,particle_id,tx,ty,tz,tpx,tpy,tpz,weight
count,120939.00000,1.209390e+05,120939.000000,120939.000000,120939.000000,120939.000000,120939.000000,120939.000000,120939.000000
mean,60470.00000,4.103431e+17,-1.767847,4.754207,-2.309751,-528.415039,437.339355,-17.295870,0.000008
std,34912.22644,3.231682e+17,305.709686,305.167542,1061.935425,188079.062500,189807.906250,272778.718750,0.000008
min,1.00000,0.000000e+00,-1024.839966,-1025.099976,-2955.500000,-999841.000000,-999861.000000,-1000000.000000,0.000000
25%,30235.50000,1.035873e+17,-100.141998,-95.873302,-654.862000,-0.329628,-0.309520,-2.038540,0.000004
50%,60470.00000,3.648081e+17,-1.651420,1.012140,-1.759190,-0.006623,0.006257,-0.007396,0.000007
75%,90704.50000,7.115699e+17,97.251553,103.222000,655.110504,0.308269,0.329654,2.038135,0.000011
max,120939.00000,9.683079e+17,1025.349976,1024.849976,2955.500000,999910.000000,999798.000000,1000000.000000,0.000165
